# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Goals
Create a humidity heat map bassed on data from previous studies.

Identify places to vacation bassed on ideal temprature cloudcover and humidity

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json



#    Need for slowing down the query loop and date stamps
import time

# Import API key
from api_keys import g_key

In [2]:
class GooglePlaces(object):
    def __init__(self, apiKey):
        super(GooglePlaces, self).__init__()
        self.apiKey = apiKey

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
#weather_data_df = pd.read_csv("big_weather_data_df.csv")

#works
#weather_data_o112 = pd.read_csv("../output_data/com_weather_data_df.csv")
#weather_data_df = weather_data_o112
#weather_data_o112
weather_data_df = pd.read_csv("output_2/big_weather_data_df.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
 # Store latitude and longitude in locations
locations = weather_data_df[["Latitude", "Longitude"]]

#extra import probably delete
# Fill NaN values and convert to float
humid = weather_data_df["Humidity (RH)%"].astype(float)

In [5]:
gmaps.configure(api_key = g_key)

In [6]:
# Plot Heatmap
humid_fig = gmaps.figure(center = [0,0],zoom_level = 2)


# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid , 
                                 dissipating=False, max_intensity=65,
                                 point_radius=2)


# Add layer
humid_fig.add_layer(heat_layer)

#label map


# Display figure
humid_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * Drop any rows will null values.

    * A max temperature lower than 80 degrees but higher than 70.
    * Wind speed less than 10 mph.
    * Zero cloudiness.
    * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

Note: Feel free to adjust to your specifications but be sure to limit the number of rows returned by your API requests to a reasonable number.

In [7]:
#weather_data_df

In [8]:
mod_temp_df = weather_data_df.loc[(weather_data_df["Temp F"] < 80) & (weather_data_df["Temp F"] > 70) & 
                                  (weather_data_df["wind speed (mph)"] < 6) &
                                  (weather_data_df["Cloud% Cover"] == 0)].copy()

In [9]:
mod_temp_df

,Unnamed: 0,City ID,City,Country Code,Latitude,Longitude,Temp Kelvin,Temp C,Temp F,Humidity (RH)%,Cloud% Cover,wind speed (m/s),wind speed (mph),Time stamp
190,190,315468,Ergani,TR,38.27,39.76,299.10,26.0,79.0,36,0,2.10,4.67,['21 Oct 2020 09:01:15']
309,309,2094027,Kieta,PG,-6.22,155.63,299.44,26.0,79.0,77,0,1.46,3.24,['21 Oct 2020 09:01:15']
374,374,1062553,Mahanoro,MG,-19.90,48.80,297.58,24.0,76.0,64,0,2.34,5.20,['21 Oct 2020 09:01:15']


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

# Uses google call to get hotel data

In [10]:
# Identifies Hotels, address, and longitude and latitude of the Hotel
### works well

Hotel_Name = []
hotel_address = []
hotel_rating = []

hotel_lat = []
hotel_lon = []

hotel_addresCC = []


# params dictionary to update each iteration
params = {
            "radius": 50000,
            "types": "lodging",
            "keyword": "hotel",
            "key": g_key
}

# Use the lat/lng we recovered to identify airports

for index, row in mod_temp_df.iterrows():
    

    
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    
#make request and print url

    name_address = requests.get(base_url, params=params)
    #print(name_address)

########     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)
 #########   # convert to json
    name_address = name_address.json()
#    print(json.dumps(name_address, indent=4, sort_keys=True))


    
    
    try:
####################   Stratagy 1 gets a new datafame based on hotel. Prefered method
###################    Works well but can't find a way to get country and city data from json
####################   Cant revese look up but working on it

        #list stratagy for new data frame does not have country
        Hotel_Name.append(name_address["results"][0]["name"])
        hotel_address.append(name_address["results"][0]["vicinity"])
        hotel_rating.append(name_address["results"][0]["rating"])
        
        hotel_lat.append(name_address["results"][0]["geometry"]["location"]["lat" ])
        hotel_lon.append(name_address["results"][0]["geometry"]["location"]["lng"])
        
        hotel_addresCC.append(name_address["results"][0]["plus_code"]["compound_code"])
        
    
    except (KeyError, IndexError):
        print("Hotel not found")


In [11]:
#chec hotel name list probably delete
print(Hotel_Name)

['Radisson Blu Hotel Diyarbakir', 'Rising Sun Lodge', 'Bungalow le Prestige']


In [38]:
# zips hotel dataframe
hotel_df = pd.DataFrame(list(zip(Hotel_Name, hotel_address, hotel_rating, hotel_lat, hotel_lon, hotel_addresCC)), 
                    columns =["Nearest Hotel", "Hotel Address", "Hotel Rating", "Lat", "Lng", "GarCountry"])
hotel_df

,Nearest Hotel,Hotel Address,Hotel Rating,Lat,Lng,GarCountry
0,Radisson Blu Hotel Diyarbakir,"Fırat, Şanlıurfa Blv. No 170, Kayapınar",4.4,37.921952,40.147579,"W4CX+Q2 Kayapınar, Diyarbakır, Turkey"
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209,"QH88+CF Arawa, Papua New Guinea"
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832,"3RW7+H4 Mahanoro, Madagascar"


In [39]:
hotel_df[['garbage','City','Country']] = hotel_df["GarCountry"].str.split(',',expand=True)
hotel_df

,Nearest Hotel,Hotel Address,Hotel Rating,Lat,Lng,GarCountry,garbage,City,Country
0,Radisson Blu Hotel Diyarbakir,"Fırat, Şanlıurfa Blv. No 170, Kayapınar",4.4,37.921952,40.147579,"W4CX+Q2 Kayapınar, Diyarbakır, Turkey",W4CX+Q2 Kayapınar,Diyarbakır,Turkey
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209,"QH88+CF Arawa, Papua New Guinea",QH88+CF Arawa,Papua New Guinea,None
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832,"3RW7+H4 Mahanoro, Madagascar",3RW7+H4 Mahanoro,Madagascar,None


In [40]:
hotel_df = hotel_df.drop(["GarCountry"], axis=1)


In [41]:
hotel_df = hotel_df.drop(["garbage"], axis=1)
hotel_df = hotel_df.copy()

In [42]:
hotel_df

,Nearest Hotel,Hotel Address,Hotel Rating,Lat,Lng,City,Country
0,Radisson Blu Hotel Diyarbakir,"Fırat, Şanlıurfa Blv. No 170, Kayapınar",4.4,37.921952,40.147579,Diyarbakır,Turkey
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209,Papua New Guinea,None
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832,Madagascar,None


In [ ]:
hotel_df

# Final dataframe for graphing

In [52]:
hotel_df

,Nearest Hotel,Hotel Address,Hotel Rating,Lat,Lng,City,Country
0,Radisson Blu Hotel Diyarbakir,"Fırat, Şanlıurfa Blv. No 170, Kayapınar",4.4,37.921952,40.147579,Diyarbakır,Turkey
1,Rising Sun Lodge,"Section 16, Lot 36,, Arawa",3.7,-6.233893,155.566209,Papua New Guinea,None
2,Bungalow le Prestige,Ambalamangahazo,4.0,-19.903613,48.812832,Madagascar,None


# Graphs

In [53]:
 # Store latitude and longitude in locations
locations = hotel_df[["Lat", "Lng"]]

# Fill NaN values and convert to float
rating = hotel_df["Hotel Rating"].astype(float)

In [54]:
# Purpose: a clean plot
# Create a map using state centroid coordinates to set markers
marker_locations = hotel_df[['Lat', 'Lng']]

# Create a marker_layer using the poverty list to fill the info box
fig_pin = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,
    info_box_content=[f"Hotel: {Hotel_Name}" for hotel in Hotel_Name])
fig_pin.add_layer(markers)
fig_pin

Figure(layout=FigureLayout(height='420px'))

# Main Hotel plot bassed on humidity heat map

In [55]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {Hotel_Name}" for hotel in Hotel_Name])
humid_fig.add_layer(markers)


# Display Map


humid_fig


Figure(layout=FigureLayout(height='420px'))

#  Conclusion
Identifed three interesting places to go on vacation.